<a href="https://colab.research.google.com/github/clee24/JIS/blob/main/AlphaPose_JIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of required python library

In [ ]:
! pip install pyyaml
! pip install scipy
! pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113
! pip install pillow


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/731.7 MB 2.5 MB/s eta 0:03:23

### Importing required modules

In [ ]:
import torch
import yaml, scipy, os

### Verify package versions


In [ ]:
print(torch.__version__)
print(yaml.__version__)
print(scipy.__version__)


### Downloading and setting up AlphaPose directory.

In [ ]:
# from settings import PROJECT_ROOT
!rm -rf /content/AlphaPose
os.chdir('/content/')
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

#### Install other supporting packages

In [ ]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

### Running of alphaPose setup

In [ ]:
os.chdir('/content/AlphaPose')
print(os.getcwd())
! python setup.py build develop

### Downloading pre-trained models

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

! mkdir /content/AlphaPose/detector/yolo/data
file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/yolo/data/yolov3-spp.weights')

! mkdir /content/AlphaPose/detector/tracker/data
file_id = '1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty')

file_id = '1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/pretrained_models/fast_res50_256x192.pth')



In [ ]:
!wget -P ./detector/yolox/data/ https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.0/yolox_x.pth

Mount Google drive to take input and stor output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Track Volleyball Player

*Note: Add all paths as per your google drive locations
* --video /content/drive/MyDrive/sample_video/yoga_video.mp4  //replace your video path
* --outdir /content/drive/MyDrive/sample_video/output/yoga_video  // replace your output path

In [ ]:
# MAke sure to point your input video path and output directory
os.chdir('/content/AlphaPose')
! ls
! python3 scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --video /content/drive/MyDrive/sample_video/yoga_video.mp4 --outdir /content/drive/MyDrive/sample_video/output/yoga_video --save_video --vis_fast